Starting with the file most recent file (2013 - September 10, 2018). Working with Bainbridge departures only.

In [26]:
import pandas as pd
import numpy as np

laptop_file_path = 'C:/Users/Keith/Google Drive/H Drive Full Backup/Fall 2018/CS 495 - Senior Seminar/Ferry Data/P000413-090518_2013-2018-09-10.csv'
dataset = pd.read_csv(laptop_file_path)

bainbridge_departures = dataset.loc[dataset['departure_terminal'] == 'Bainbridge'].reset_index()
bainbridge_departures = bainbridge_departures.drop('index', axis=1)

In [27]:
bainbridge_departures.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0
1,Bainbridge,NaN,45:00.0,NaN,NaN,NaN
2,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0
3,Bainbridge,NaN,20:00.0,NaN,NaN,NaN
4,Bainbridge,NaN,05:00.0,06:16.0,NaN,NaN


<a href = "https://web.archive.org/web/20130123233819/http:/www.wsdot.wa.gov/ferries/pdf/2013winter.pdf"> Winter 2013 Schedule </a>

In [28]:
# Monday-Friday leave Bainbridge Island Winter 2013
# mfw13 means Monday-Friday Winter 2013
mfw13_scheduled_departures_24h = ["0:55:00", "4:45:00", "5:20:00", "6:20:00", "7:05:00", "7:55:00","8:45:00", "9:40:00", 
                            "10:25:00", "11:30:00", "12:20:00", "13:10:00", "14:05:00", "14:55:00", "15:50:00", "16:35:00", 
                            "17:30:00", "18:30:00", "19:10:00", "20:10:00", "20:55:00", "21:45:00", "22:35:00"]

# satw13 means Saturday Winter 2013
satw13_scheduled_departures_24h = ["0:55:00","05:20:00","07:05:00","07:55:00","08:45:00","9:35:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00","0:00:00","1:25:00"]

# sunw13 means Sunday Winter 2013
sunw13_scheduled_departures_24h = ["05:20:00","07:05:00","08:45:00","9:35:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00","0:00:00","1:25:00"]

#wkw13 means Week Winder 2013
wkw13_scheduled_departures_24h = (5*mfw13_scheduled_departures_24h) + satw13_scheduled_departures_24h + sunw13_scheduled_departures_24h

In [29]:
def extractMinutesPastHour(time):
    m = time.split(':')
    return int(float(m[0]))

# For time in format hh:mm:ss this method returns mm
def extractMinutes24h(time):
    m = time.split(':')
    return int(m[1])

In [30]:
import datetime

def generate_timestamps(dataset, scheduled_departures_24h, starting_point):
    year = 2013
    month = 1
    day = 1

    tdelta = datetime.timedelta(days = 1)

    prevHour = 0

    departure_times = []
    timestamps = []
    weekdays = []

    i = starting_point

    for index, row in dataset.iterrows():
        departure24h = wkw13_scheduled_departures_24h[i % len(scheduled_departures_24h)]
        hour, minute, seconds = departure24h.split(":")
        scheduled_departure_minutes = extractMinutesPastHour(row.iloc[2])
        scheduled_departure_24h = extractMinutes24h(departure24h)

        if(int(prevHour) > int(hour)):
            d = d + tdelta
            year = d.year
            month = d.month
            day = d.day

        if (not scheduled_departure_minutes == scheduled_departure_24h):
            if(d.weekday() == 1):
                # --- this is a quick fix not sure why this works ----
                d = d - tdelta
                year = d.year
                month = d.month
                day = d.day
                # ------------------------------------

                i = i + 1

                departure24h = scheduled_departures_24h[i % len(scheduled_departures_24h)]
                scheduled_hour, scheduled_minutes, scheduled_seconds = departure24h.split(":")
                scheduled_departure_minutes = extractMinutesPastHour(row.iloc[2])

                departure_times.append(departure24h)
                d = datetime.date(year, month, day)
                timestamps.append("%s %02d:%02d:%02d" % (d, int(scheduled_hour), int(scheduled_minutes), int(scheduled_seconds)))
                weekdays.append(d.weekday())

                prevHour = hour  
                i = i + 1
                continue
                if (not scheduled_departure_minutes == scheduled_minutes):
                    print("ERROR")
                    break

            print("""Was able to match departure times up until this row at index %d:\n%s\n
                    Was trying to insert this timestamp: %s""" % 
                                              (index, row,("%s %02d:%02d:%02d" % (d, int(hour), int(minute), int(seconds)))))
            break

        departure_times.append(departure24h)
        d = datetime.date(year, month, day)
        timestamps.append("%s %02d:%02d:%02d" % (d, int(hour), int(minute), int(seconds)))
        weekdays.append(d.weekday())

        prevHour = hour
        i = i + 1
        
    return timestamps

In [31]:
timestamps = generate_timestamps(bainbridge_departures, wkw13_scheduled_departures_24h, 23)

Was able to match departure times up until this row at index 3297:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       05:00.0
actual_departure          05:30.0
Actual_Arrival            36:59.0
TransitTime                    31
Name: 3297, dtype: object

                    Was trying to insert this timestamp: 2013-05-27 05:20:00


It made it pretty far before the pattern was broken! Even past the Winter 2013 schedule.

Adding these timestamps to the dataframe we have

In [32]:
bainbridge_departures_with_timestamps = bainbridge_departures.iloc[0:3297]
bainbridge_departures_with_timestamps.index = pd.to_datetime(timestamps)

<a href = "https://stackoverflow.com/questions/40541265/convert-pandas-dataframe-column-to-np-datetime64">convert pandas dataframe column to np.datetime64 </a>

In [33]:
# Bainbridge Winter 2013 (Starting January 1, 2013 - March 23, 2013)
bainbrigdew13 = bainbridge_departures_with_timestamps["2013-01-01" : "2013-03-23"]
bainbrigdew13.shape

(1851, 6)

In [34]:
bainbrigdew13.tail()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2013-03-23 19:10:00,Bainbridge,Colman,10:00.0,32:08.0,03:29.0,31.0
2013-03-23 20:10:00,Bainbridge,Colman,10:00.0,10:39.0,42:37.0,32.0
2013-03-23 20:55:00,Bainbridge,Colman,55:00.0,05:16.0,36:54.0,31.0
2013-03-23 21:45:00,Bainbridge,Colman,45:00.0,45:56.0,18:06.0,33.0
2013-03-23 22:30:00,Bainbridge,Colman,30:00.0,31:30.0,03:31.0,32.0


Now lets try <a href = "https://web.archive.org/web/20130624071202/http:/www.wsdot.wa.gov/ferries/pdf/2013Spring.pdf"> Spring 2013 </a>

In [37]:
# Cutting out winter 2013 from bainbridge_departures
bainbridge_departures_nw13 = bainbridge_departures.iloc[1851:]

In [38]:
bainbridge_departures_nw13.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
1851,Bainbridge,Colman,00:00.0,00:00.0,31:03.0,31.0
1852,Bainbridge,Colman,25:00.0,28:00.0,59:24.0,31.0
1853,Bainbridge,Colman,20:00.0,21:27.0,53:07.0,32.0
1854,Bainbridge,Colman,05:00.0,05:10.0,36:45.0,31.0
1855,Bainbridge,Colman,45:00.0,45:00.0,17:25.0,32.0


Spring 2013 schedule is effective Sunday, March 24, 2013 - Saturday, June 15, 2013

In [ ]:
# All are times departing Bainbridge Island Spring 2013

# mfs13 means Monday-Friday Spring 2013
mfs13_scheduled_departures_24h = ["0:55:00", "4:45:00", "5:20:00", "6:20:00", "7:05:00", "7:55:00","8:45:00", "9:40:00", 
                            "10:25:00", "11:30:00", "12:20:00", "13:10:00", "14:05:00", "14:55:00", "15:50:00", "16:35:00", 
                            "17:30:00", "18:30:00", "19:10:00", "20:10:00", "20:55:00", "21:45:00", "22:35:00"]

# sats13 means Saturday Spring 2013
sats13_scheduled_departures_24h = ["0:55:00","05:20:00","07:05:00","07:55:00","08:45:00","9:35:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00","0:00:00","1:25:00"]

# may27s13 means may 27 Spring 2013
may27s13_scheduled_departures_24h = sats13_scheduled_departures_24h

# suns13 means Sunday Spring 2013
suns13_scheduled_departures_24h = ["05:20:00","07:05:00","08:45:00","9:35:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00","0:00:00","1:25:00"]

#s13 means Spring 2013
s13_scheduled_departures_24h = (5*mfw13_scheduled_departures_24h) + satw13_scheduled_departures_24h + sunw13_scheduled_departures_24h